In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install rich

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 237 kB 5.1 MB/s 
     |████████████████████████████████| 51 kB 6.5 MB/s 


In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 54.7 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import os
import re
import rich  # 출력을 예쁘게 꾸며주는 라이브러리
from rich.table import Table

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from tqdm.auto import tqdm

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

#주최측 공식 데이터
#train_df = pd.read_csv('/content/gdrive/My Drive/kerc/data/tmp/df_train_extended.tsv',sep = '\t')
train_df = pd.read_csv('/content/gdrive/My Drive/kerc/data/SGwannabe_train_set_4403.csv')
train_labels_df = pd.read_csv('/content/gdrive/My Drive/kerc/data/train_labels.csv')
test_df = pd.read_csv('/content/gdrive/My Drive/kerc/data/tmp/df_public_test_extended.tsv', sep = '\t')
#test_df_2 = pd.read_csv('/content/gdrive/My Drive/kerc/data/df_private_test_extended.tsv', sep = '\t')

#lee 데이터
# train_df = pd.read_csv('/content/gdrive/My Drive/kerc/data/lee/df_train_extended.tsv',sep = '\t')
# train_labels_df = pd.read_csv('/content/gdrive/My Drive/kerc/data/lee/train_labels.csv')
# test_df = pd.read_csv('/content/gdrive/My Drive/kerc/data/lee/df_public_test_extended.tsv', sep = '\t')


#submission = pd.read_csv('/content/gdrive/My Drive/kerc/data/sample_submission.csv')




In [ ]:
test_df.head()

,sentence_id,person,sentence,scene,context,target_speaker_ctx,other_speaker_ctx,scene_sents
0,10,현찰,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",,,
1,11,현찰,"계약할 서류는 태연희실장 통해서 보내 드렸는데 확인해 보셨죠? 예, 예.",S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.,,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.
2,12,경위1,김순경 경사님 사모님 아니십니까?,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다. 계약할 서류는 태연희실...,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.계약할 서류는 태연희실장...
3,13,과자,아 예.,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다. 계약할 서류는 태연희실...,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.계약할 서류는 태연희실장...
4,14,경위1,저 기억하시죠? 00지구대에서 김경사님 모시고 근무했었습니다.,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",김순경 경사님 사모님 아니십니까?,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다. 계약할 서류는 태연희실...,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.계약할 서류는 태연희실장...


In [ ]:
test_df_2.head()

,sentence_id,person,sentence,scene,context,target_speaker_ctx,other_speaker_ctx,scene_sents
0,55,마탄,"팀장님 인기 죽입니다. 바쁘신데 제가 대신 관리를,",S0118,들오는 이상. 꽃다발 여기저기,,,
1,56,이상,"그만 띄우고, 서장님께 보고드릴 보고서 준비해.",S0118,들오는 이상. 꽃다발 여기저기,,"팀장님 인기 죽입니다. 바쁘신데 제가 대신 관리를,","팀장님 인기 죽입니다. 바쁘신데 제가 대신 관리를,"
2,57,마탄,"이미 준비 싹, 한줄 알았더니 바로 끝내러 가겠습니다. 충성",S0118,들오는 이상. 꽃다발 여기저기,"팀장님 인기 죽입니다. 바쁘신데 제가 대신 관리를,","그만 띄우고, 서장님께 보고드릴 보고서 준비해.","팀장님 인기 죽입니다. 바쁘신데 제가 대신 관리를,그만 띄우고, 서장님께 보고드릴..."
3,58,도우미,오늘 못가서 죄송해요. 멋진 도련님 모습 보고 싶었는데. 대신 맛있는거 해놀테니까 ...,S0118,들오는 이상. 꽃다발 여기저기,,"팀장님 인기 죽입니다. 바쁘신데 제가 대신 관리를, 그만 띄우고, 서장님께 보고드...","팀장님 인기 죽입니다. 바쁘신데 제가 대신 관리를,그만 띄우고, 서장님께 보고드릴..."
4,59,이상,형수님!,S0118,들오는 이상. 꽃다발 여기저기,"그만 띄우고, 서장님께 보고드릴 보고서 준비해.","팀장님 인기 죽입니다. 바쁘신데 제가 대신 관리를, 이미 준비 싹, 한줄 알았더니...","팀장님 인기 죽입니다. 바쁘신데 제가 대신 관리를,그만 띄우고, 서장님께 보고드릴..."


In [ ]:
test_df = test_df.append(test_df_2)

In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2566 entries, 0 to 2565
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   sentence_id         2566 non-null   int64 
 1   person              2566 non-null   object
 2   sentence            2566 non-null   object
 3   scene               2566 non-null   object
 4   context             1795 non-null   object
 5   target_speaker_ctx  2566 non-null   object
 6   other_speaker_ctx   2566 non-null   object
 7   scene_sents         2566 non-null   object
dtypes: int64(1), object(7)
memory usage: 160.5+ KB


In [ ]:
test_df.head()

,sentence_id,person,sentence,scene,context,target_speaker_ctx,other_speaker_ctx,scene_sents
0,10,현찰,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",,,
1,11,현찰,"계약할 서류는 태연희실장 통해서 보내 드렸는데 확인해 보셨죠? 예, 예.",S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.,,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.
2,12,경위1,김순경 경사님 사모님 아니십니까?,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다. 계약할 서류는 태연희실...,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.계약할 서류는 태연희실장...
3,13,과자,아 예.,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다. 계약할 서류는 태연희실...,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.계약할 서류는 태연희실장...
4,14,경위1,저 기억하시죠? 00지구대에서 김경사님 모시고 근무했었습니다.,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",김순경 경사님 사모님 아니십니까?,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다. 계약할 서류는 태연희실...,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.계약할 서류는 태연희실장...


In [ ]:
validation_df = pd.read_csv('/content/gdrive/My Drive/kerc/data/SGwannabe_validation_set_2936.csv')


In [ ]:
train_labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7339 entries, 0 to 7338
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sentence_id  7339 non-null   int64 
 1   label        7339 non-null   object
dtypes: int64(1), object(1)
memory usage: 114.8+ KB


In [ ]:
#train_df = pd.merge(train_df, train_labels_df, on = 'sentence_id', how = 'left')

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4403 entries, 0 to 4402
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   sentence_id         4403 non-null   int64 
 1   person              4403 non-null   object
 2   sentence            4403 non-null   object
 3   scene               4403 non-null   object
 4   context             3194 non-null   object
 5   target_speaker_ctx  4403 non-null   object
 6   other_speaker_ctx   4403 non-null   object
 7   scene_sents         4403 non-null   object
 8   label               4403 non-null   object
dtypes: int64(1), object(8)
memory usage: 309.7+ KB


In [ ]:
validation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2936 entries, 0 to 2935
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   sentence_id         2936 non-null   int64 
 1   person              2936 non-null   object
 2   sentence            2936 non-null   object
 3   scene               2936 non-null   object
 4   context             2128 non-null   object
 5   target_speaker_ctx  2936 non-null   object
 6   other_speaker_ctx   2936 non-null   object
 7   scene_sents         2936 non-null   object
 8   label               2936 non-null   object
dtypes: int64(1), object(8)
memory usage: 206.6+ KB


In [ ]:
train_df.head()

,sentence_id,person,sentence,scene,context,target_speaker_ctx,other_speaker_ctx,scene_sents,label
0,11993,상태,엄마 왜 종남이만 많이 줘? 내거야.,S3525,상태보면 종남접시에 좀더 많이 담긴,,"자, 똑같이 나눴으니까 싸우지들 말고 먹어.","자, 똑같이 나눴으니까 싸우지들 말고 먹어.",dysphoria
1,5958,과자,또 그놈의 잔소리.,S1832,마지못해 반찬꺼내 상차리는 과자. 들오는 순경. 뒤따라 현찰,사연을 말하자면 너무 길어서 밤을 새도 모자라니 넘어갑시다. 니가 알지 내가 아냐...,아침에 해가 분명 동쪽에서 뜨던데. 오늘부터 살림연습 들어갔어? 못한다고 딱 잡아...,아침에 해가 분명 동쪽에서 뜨던데. 오늘부터 살림연습 들어갔어? 못한다고 딱 잡아...,dysphoria
2,2266,범인,너한테 무슨말 안하디?,S0759,저녁먹던중. 들어와 앉는 범인부영,언니 먹을것좀 해줘야겠다.,요새 힘든가봐요.,언니 먹을것좀 해줘야겠다.요새 힘든가봐요.,neutral
3,4564,청난,"아, 오빠만 보면 삘이팍! 온다구요.",S1457,할수없이 청난옆에 앉는 건강. 좋아서 얼른 팔짱끼는 청난. 슬쩍 빼내는 건강. 청난...,"도련님하고 저하고 삘이 통하네요. 오빠. 아버님, 오빠가 저 이렇게 했어요. 뭐요?",문부서지겠다. 그렇게 좋다니? 들어와 앉아. 예 아부지. 아 아닙니다. 내가 언제...,도련님하고 저하고 삘이 통하네요.문부서지겠다. 그렇게 좋다니?들어와 앉아.예 아부...,euphoria
4,3987,범인,태워주긴 뭘 태워줘? 걸어서 가. 부영이 앞으로 언니 신경 안쓰이게 니할일 딱딱 하...,S1326,신문보는 범인. 찻잔놓는 솔이. 이층에서 어영의 노랫소리. 슥보는 범인,벌써 나가게? 너 괜찮니? 쓸데없이. 어영아. 일루 잠깐 앉아봐. 일부러 이럴필요...,먼일인지 아주 신바람이 났시유. 예 아부지. 일찍 일어나서 운동도 하고 책상 정리...,먼일인지 아주 신바람이 났시유.벌써 나가게?예 아부지. 일찍 일어나서 운동도 하고...,dysphoria


In [ ]:
print(train_df.nunique())

sentence_id           4403
person                  76
sentence              4144
scene                  842
context                647
target_speaker_ctx    3170
other_speaker_ctx     3648
scene_sents           3848
label                    3
dtype: int64


In [ ]:
print(train_df['label'].value_counts())

dysphoria    2715
euphoria     1180
neutral       508
Name: label, dtype: int64


In [ ]:
train_df = train_df.replace(np.nan, '', regex=True)

In [ ]:
test_df = test_df.replace(np.nan, '', regex=True)

In [ ]:
validation_df = validation_df.replace(np.nan, '', regex=True)

In [ ]:
train_df.head()

,sentence_id,person,sentence,scene,context,target_speaker_ctx,other_speaker_ctx,scene_sents,label
0,11993,상태,엄마 왜 종남이만 많이 줘? 내거야.,S3525,상태보면 종남접시에 좀더 많이 담긴,,"자, 똑같이 나눴으니까 싸우지들 말고 먹어.","자, 똑같이 나눴으니까 싸우지들 말고 먹어.",dysphoria
1,5958,과자,또 그놈의 잔소리.,S1832,마지못해 반찬꺼내 상차리는 과자. 들오는 순경. 뒤따라 현찰,사연을 말하자면 너무 길어서 밤을 새도 모자라니 넘어갑시다. 니가 알지 내가 아냐...,아침에 해가 분명 동쪽에서 뜨던데. 오늘부터 살림연습 들어갔어? 못한다고 딱 잡아...,아침에 해가 분명 동쪽에서 뜨던데. 오늘부터 살림연습 들어갔어? 못한다고 딱 잡아...,dysphoria
2,2266,범인,너한테 무슨말 안하디?,S0759,저녁먹던중. 들어와 앉는 범인부영,언니 먹을것좀 해줘야겠다.,요새 힘든가봐요.,언니 먹을것좀 해줘야겠다.요새 힘든가봐요.,neutral
3,4564,청난,"아, 오빠만 보면 삘이팍! 온다구요.",S1457,할수없이 청난옆에 앉는 건강. 좋아서 얼른 팔짱끼는 청난. 슬쩍 빼내는 건강. 청난...,"도련님하고 저하고 삘이 통하네요. 오빠. 아버님, 오빠가 저 이렇게 했어요. 뭐요?",문부서지겠다. 그렇게 좋다니? 들어와 앉아. 예 아부지. 아 아닙니다. 내가 언제...,도련님하고 저하고 삘이 통하네요.문부서지겠다. 그렇게 좋다니?들어와 앉아.예 아부...,euphoria
4,3987,범인,태워주긴 뭘 태워줘? 걸어서 가. 부영이 앞으로 언니 신경 안쓰이게 니할일 딱딱 하...,S1326,신문보는 범인. 찻잔놓는 솔이. 이층에서 어영의 노랫소리. 슥보는 범인,벌써 나가게? 너 괜찮니? 쓸데없이. 어영아. 일루 잠깐 앉아봐. 일부러 이럴필요...,먼일인지 아주 신바람이 났시유. 예 아부지. 일찍 일어나서 운동도 하고 책상 정리...,먼일인지 아주 신바람이 났시유.벌써 나가게?예 아부지. 일찍 일어나서 운동도 하고...,dysphoria


In [ ]:
test_df.head()

,sentence_id,person,sentence,scene,context,target_speaker_ctx,other_speaker_ctx,scene_sents
0,10,현찰,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",,,
1,11,현찰,"계약할 서류는 태연희실장 통해서 보내 드렸는데 확인해 보셨죠? 예, 예.",S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.,,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.
2,12,경위1,김순경 경사님 사모님 아니십니까?,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다. 계약할 서류는 태연희실...,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.계약할 서류는 태연희실장...
3,13,과자,아 예.,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다. 계약할 서류는 태연희실...,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.계약할 서류는 태연희실장...
4,14,경위1,저 기억하시죠? 00지구대에서 김경사님 모시고 근무했었습니다.,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",김순경 경사님 사모님 아니십니까?,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다. 계약할 서류는 태연희실...,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.계약할 서류는 태연희실장...


In [ ]:
# 두 칸 이상의 빈칸을 한칸으로 바꿔줌
#train_df["sentence"]  = train_df["sentence"].str.replace("\s+", " ", regex=True)+" "+train_df["context"].str.replace("\s+", " ", regex=True)+" "+train_df["target_speaker_ctx"].str.replace("\s+", " ", regex=True)+" "+train_df["other_speaker_ctx"].str.replace("\s+", " ", regex=True)+" "+train_df["scene_sents"].str.replace("\s+", " ", regex=True)
#train_df["sentence"]  = train_df["sentence"].str.replace("\s+", " ", regex=True)+" "+train_df["scene_sents"].str.replace("\s+", " ", regex=True)
train_df["sentence"]  = train_df["sentence"].str.replace("\s+", " ", regex=True)+" "+train_df["context"].str.replace("\s+", " ", regex=True)+" "+train_df["target_speaker_ctx"].str.replace("\s+", " ", regex=True)+" "+train_df["other_speaker_ctx"].str.replace("\s+", " ", regex=True)

In [ ]:
#test_df["sentence"]  = test_df["sentence"].str.replace("\s+", " ", regex=True)+" "+test_df["context"].str.replace("\s+", " ", regex=True)+" "+test_df["target_speaker_ctx"].str.replace("\s+", " ", regex=True)+" "+test_df["other_speaker_ctx"].str.replace("\s+", " ", regex=True)+" "+test_df["scene_sents"].str.replace("\s+", " ", regex=True)
#test_df["sentence"]  = test_df["sentence"].str.replace("\s+", " ", regex=True)+" "+test_df["scene_sents"].str.replace("\s+", " ", regex=True)
test_df["sentence"]  = test_df["sentence"].str.replace("\s+", " ", regex=True)+" "+test_df["context"].str.replace("\s+", " ", regex=True)+" "+test_df["target_speaker_ctx"].str.replace("\s+", " ", regex=True)+" "+test_df["other_speaker_ctx"].str.replace("\s+", " ", regex=True)

In [ ]:
validation_df["sentence"]  = validation_df["sentence"].str.replace("\s+", " ", regex=True)+" "+validation_df["context"].str.replace("\s+", " ", regex=True)+" "+validation_df["target_speaker_ctx"].str.replace("\s+", " ", regex=True)+" "+validation_df["other_speaker_ctx"].str.replace("\s+", " ", regex=True)
#validation_df["sentence"]  = validation_df["sentence"].str.replace("\s+", " ", regex=True)+" "+validation_df["context"].str.replace("\s+", " ", regex=True)+" "+validation_df["target_speaker_ctx"].str.replace("\s+", " ", regex=True)+" "+validation_df["other_speaker_ctx"].str.replace("\s+", " ", regex=True)+" "+validation_df["scene_sents"].str.replace("\s+", " ", regex=True)


In [ ]:
def preprocess(text):
  text = text.lower()
  
  text = re.sub("[^A-Za-z가-힣 ]","", text)  # re.sub(r"[^A-Za-zㄱ-ㅎㅏ-ㅣ가-힣 ]","", text)
  return text

train_df['preprocessed'] = train_df.sentence.apply(lambda x : preprocess(x))
test_df['preprocessed'] = test_df.sentence.apply(lambda x : preprocess(x))
validation_df['preprocessed'] = validation_df.sentence.apply(lambda x : preprocess(x))

train_df.head()

,sentence_id,person,sentence,scene,context,target_speaker_ctx,other_speaker_ctx,scene_sents,label,preprocessed
0,11993,상태,"엄마 왜 종남이만 많이 줘? 내거야. 상태보면 종남접시에 좀더 많이 담긴 자,...",S3525,상태보면 종남접시에 좀더 많이 담긴,,"자, 똑같이 나눴으니까 싸우지들 말고 먹어.","자, 똑같이 나눴으니까 싸우지들 말고 먹어.",dysphoria,엄마 왜 종남이만 많이 줘 내거야 상태보면 종남접시에 좀더 많이 담긴 자 똑같...
1,5958,과자,또 그놈의 잔소리. 마지못해 반찬꺼내 상차리는 과자. 들오는 순경. 뒤따라 현찰 ...,S1832,마지못해 반찬꺼내 상차리는 과자. 들오는 순경. 뒤따라 현찰,사연을 말하자면 너무 길어서 밤을 새도 모자라니 넘어갑시다. 니가 알지 내가 아냐...,아침에 해가 분명 동쪽에서 뜨던데. 오늘부터 살림연습 들어갔어? 못한다고 딱 잡아...,아침에 해가 분명 동쪽에서 뜨던데. 오늘부터 살림연습 들어갔어? 못한다고 딱 잡아...,dysphoria,또 그놈의 잔소리 마지못해 반찬꺼내 상차리는 과자 들오는 순경 뒤따라 현찰 사연을...
2,2266,범인,너한테 무슨말 안하디? 저녁먹던중. 들어와 앉는 범인부영 언니 먹을것좀 해줘야겠다...,S0759,저녁먹던중. 들어와 앉는 범인부영,언니 먹을것좀 해줘야겠다.,요새 힘든가봐요.,언니 먹을것좀 해줘야겠다.요새 힘든가봐요.,neutral,너한테 무슨말 안하디 저녁먹던중 들어와 앉는 범인부영 언니 먹을것좀 해줘야겠다 ...
3,4564,청난,"아, 오빠만 보면 삘이팍! 온다구요. 할수없이 청난옆에 앉는 건강. 좋아서 얼른 팔...",S1457,할수없이 청난옆에 앉는 건강. 좋아서 얼른 팔짱끼는 청난. 슬쩍 빼내는 건강. 청난...,"도련님하고 저하고 삘이 통하네요. 오빠. 아버님, 오빠가 저 이렇게 했어요. 뭐요?",문부서지겠다. 그렇게 좋다니? 들어와 앉아. 예 아부지. 아 아닙니다. 내가 언제...,도련님하고 저하고 삘이 통하네요.문부서지겠다. 그렇게 좋다니?들어와 앉아.예 아부...,euphoria,아 오빠만 보면 삘이팍 온다구요 할수없이 청난옆에 앉는 건강 좋아서 얼른 팔짱끼는 ...
4,3987,범인,태워주긴 뭘 태워줘? 걸어서 가. 부영이 앞으로 언니 신경 안쓰이게 니할일 딱딱 하...,S1326,신문보는 범인. 찻잔놓는 솔이. 이층에서 어영의 노랫소리. 슥보는 범인,벌써 나가게? 너 괜찮니? 쓸데없이. 어영아. 일루 잠깐 앉아봐. 일부러 이럴필요...,먼일인지 아주 신바람이 났시유. 예 아부지. 일찍 일어나서 운동도 하고 책상 정리...,먼일인지 아주 신바람이 났시유.벌써 나가게?예 아부지. 일찍 일어나서 운동도 하고...,dysphoria,태워주긴 뭘 태워줘 걸어서 가 부영이 앞으로 언니 신경 안쓰이게 니할일 딱딱 하고 ...


In [ ]:
from konlpy import tag
from konlpy.tag import Okt
ok = Okt()
main_pos = [] # 데이터프레임의 새 컬럼이 될 리스트
for sentence in train_df['preprocessed']: # 리뷰들을 하나씩 가져옵니다
    pos = ok.pos(sentence) # 형태소 분석을 진행하고 해당 리스트를 pos라는 변수로 받습니다
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] not in ("Josa","Suffix")] # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가합니다.
    main_words_str = " ".join(main_words) # main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    main_pos.append(main_words_str) # 선택한 형태소들로 이루어진 문자열을 최종 리스트에 추가해줍니다

train_df["preprocessed"] = pd.DataFrame(main_pos) # 리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가해줍니다.

In [ ]:
main_pos = [] # 데이터프레임의 새 컬럼이 될 리스트
for sentence in test_df['preprocessed']: # 리뷰들을 하나씩 가져옵니다
    pos = ok.pos(sentence) # 형태소 분석을 진행하고 해당 리스트를 pos라는 변수로 받습니다
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] not in ("Josa","Suffix")] # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가합니다.
    main_words_str = " ".join(main_words) # main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    main_pos.append(main_words_str) # 선택한 형태소들로 이루어진 문자열을 최종 리스트에 추가해줍니다

test_df["preprocessed"] = pd.DataFrame(main_pos) # 리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가해줍니다.

In [ ]:
main_pos = [] # 데이터프레임의 새 컬럼이 될 리스트
for sentence in validation_df['preprocessed']: # 리뷰들을 하나씩 가져옵니다
    pos = ok.pos(sentence) # 형태소 분석을 진행하고 해당 리스트를 pos라는 변수로 받습니다
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] not in ("Josa","Suffix")] # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가합니다.
    main_words_str = " ".join(main_words) # main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    main_pos.append(main_words_str) # 선택한 형태소들로 이루어진 문자열을 최종 리스트에 추가해줍니다

validation_df["preprocessed"] = pd.DataFrame(main_pos) # 리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가해줍니다.

In [ ]:
# #불용어도 제거해보자
# with open('/content/gdrive/My Drive/kerc/data/한국어불용어100.txt') as f:
#     lines = f.readlines()
# stopwords = []
# for stopword in lines:
#     stopwords.append(stopword.split('\t')[0])
# print(stopwords)

['이', '있', '하', '것', '들', '그', '되', '수', '이', '보', '않', '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', '년', '가', '한', '지', '대하', '오', '말', '일', '그렇', '위하', '때문', '그것', '두', '말하', '알', '그러나', '받', '못하', '일', '그런', '또', '문제', '더', '사회', '많', '그리고', '좋', '크', '따르', '중', '나오', '가지', '씨', '시키', '만들', '지금', '생각하', '그러', '속', '하나', '집', '살', '모르', '적', '월', '데', '자신', '안', '어떤', '내', '내', '경우', '명', '생각', '시간', '그녀', '다시', '이런', '앞', '보이', '번', '나', '다른', '어떻', '여자', '개', '전', '들', '사실', '이렇', '점', '싶', '말', '정도', '좀', '원', '잘', '통하', '소리', '놓']


In [ ]:
# train_df['preprocessed'] = train_df['preprocessed'].apply(lambda x: ' '.join([w for w in x.split() if w not in stopwords]))
# test_df['preprocessed'] = test_df['preprocessed'].apply(lambda x: ' '.join([w for w in x.split() if w not in stopwords]))
# validation_df['preprocessed'] = validation_df['preprocessed'].apply(lambda x: ' '.join([w for w in x.split() if w not in stopwords]))

In [ ]:
#등장인물 발화 문장 개수
train_df['person'].value_counts()

이상     567
어영     473
우미     430
과자     426
건강     382
      ... 
친구들      1
여주인      1
여손님      1
부장       1
친구       1
Name: person, Length: 76, dtype: int64

In [ ]:
test_df['person'].value_counts()

이상        598
과자        522
어영        497
우미        428
현찰        417
         ... 
우미현찰애들      1
사내방송        1
검사보         1
여자2         1
검대          1
Name: person, Length: 75, dtype: int64

In [ ]:
#등장인물 발화 언급시 다 replace 시킴
list_merge = []
list_merge.extend(train_df['person'].unique())
list_merge.extend(test_df['person'].unique())
list_merge.extend(validation_df['person'].unique())

for idx in range(len(list_merge)):
  person = list_merge[idx]
  print(person)
  train_df=train_df.replace(person,'')
  test_df=test_df.replace(person,'')
  validation_df=validation_df.replace(person,'')

상태
과자
범인
청난
우미
솔이
박사장
이상
현찰
어영
건강
연희
치장
마탄
순경
재수
범용
부영
경사
성미
난자
도망자
서장
장사
유치장
혼수
주인
직원
태백
종업원
구대
관장
부장
종남
조세기
원장
기사
부동산
사중
점쟁이
성미모
카드남
혼수상태
지구대
여손님
여주인
장사장
건강현찰
영교
의사
방범용
친구들
일동
우미어영
동기
이상현찰
부장검사
대장
친구1
웨이터
친1
조사중
검사보
현찰이상
이장
남자1
하행선
손주
사장
할머니
도우미
이상어영
아줌
아1
간호사
친구
현찰
경위1
과자
순경
사회자
우미
방범용
유치장
이상
건강
주범인
부영
범인
어영
구청장
구의원
부인1
부인2
부인
솔이
관장
재수
종업원
지구대
연희
직원
마탄
청난
유치
혼수
상태
간호사
범용
치장
사중
성미
성미모
선녀
일동
딴직원
혼수상태
난자
지점장
종남
행선
사내방송
태백
사내1
검사보
서장
경찰
여자1
여자2
구대
탈주범1
박사장
원장
아줌마
우미현찰애들
연희
재수
범인
순경
건강
청난
부영
과자
우미
구급대원
어영
이상
혼수상태
현찰
남자
서장
대장
난자
솔이
주인
치장
동기
범용
태백
종업원
구대
마탄
종남
지구대
박사장
성미
기사
조세기
하행선
유치장
현찰이상
직원
조사중
혼수
아2
방범용
일동
할머니
친구
관장
원장
사내1
사중
장사장
카드남
아줌마2
장사
사장
간호사
아줌
이상어영
영교
성미모
여손님
지성미
여자
상태
손님
경사
친구2
운전자
남자1
아줌마1


In [ ]:
train_df.head()

,sentence_id,person,sentence,scene,context,target_speaker_ctx,other_speaker_ctx,scene_sents,label,preprocessed
0,11993,,"엄마 왜 종남이만 많이 줘? 내거야. 상태보면 종남접시에 좀더 많이 담긴 자,...",S3525,상태보면 종남접시에 좀더 많이 담긴,,"자, 똑같이 나눴으니까 싸우지들 말고 먹어.","자, 똑같이 나눴으니까 싸우지들 말고 먹어.",dysphoria,엄마 왜 종남이만 많이 줘 내거야 상태보면 종남접시에 좀더 많이 담긴 자 똑같...
1,5958,,또 그놈의 잔소리. 마지못해 반찬꺼내 상차리는 과자. 들오는 순경. 뒤따라 현찰 ...,S1832,마지못해 반찬꺼내 상차리는 과자. 들오는 순경. 뒤따라 현찰,사연을 말하자면 너무 길어서 밤을 새도 모자라니 넘어갑시다. 니가 알지 내가 아냐...,아침에 해가 분명 동쪽에서 뜨던데. 오늘부터 살림연습 들어갔어? 못한다고 딱 잡아...,아침에 해가 분명 동쪽에서 뜨던데. 오늘부터 살림연습 들어갔어? 못한다고 딱 잡아...,dysphoria,또 그놈의 잔소리 마지못해 반찬꺼내 상차리는 과자 들오는 순경 뒤따라 현찰 사연을...
2,2266,,너한테 무슨말 안하디? 저녁먹던중. 들어와 앉는 범인부영 언니 먹을것좀 해줘야겠다...,S0759,저녁먹던중. 들어와 앉는 범인부영,언니 먹을것좀 해줘야겠다.,요새 힘든가봐요.,언니 먹을것좀 해줘야겠다.요새 힘든가봐요.,neutral,너한테 무슨말 안하디 저녁먹던중 들어와 앉는 범인부영 언니 먹을것좀 해줘야겠다 ...
3,4564,,"아, 오빠만 보면 삘이팍! 온다구요. 할수없이 청난옆에 앉는 건강. 좋아서 얼른 팔...",S1457,할수없이 청난옆에 앉는 건강. 좋아서 얼른 팔짱끼는 청난. 슬쩍 빼내는 건강. 청난...,"도련님하고 저하고 삘이 통하네요. 오빠. 아버님, 오빠가 저 이렇게 했어요. 뭐요?",문부서지겠다. 그렇게 좋다니? 들어와 앉아. 예 아부지. 아 아닙니다. 내가 언제...,도련님하고 저하고 삘이 통하네요.문부서지겠다. 그렇게 좋다니?들어와 앉아.예 아부...,euphoria,아 오빠만 보면 삘이팍 온다구요 할수없이 청난옆에 앉는 건강 좋아서 얼른 팔짱끼는 ...
4,3987,,태워주긴 뭘 태워줘? 걸어서 가. 부영이 앞으로 언니 신경 안쓰이게 니할일 딱딱 하...,S1326,신문보는 범인. 찻잔놓는 솔이. 이층에서 어영의 노랫소리. 슥보는 범인,벌써 나가게? 너 괜찮니? 쓸데없이. 어영아. 일루 잠깐 앉아봐. 일부러 이럴필요...,먼일인지 아주 신바람이 났시유. 예 아부지. 일찍 일어나서 운동도 하고 책상 정리...,먼일인지 아주 신바람이 났시유.벌써 나가게?예 아부지. 일찍 일어나서 운동도 하고...,dysphoria,태워주긴 뭘 태워줘 걸어서 가 부영이 앞으로 언니 신경 안쓰이게 니할일 딱딱 하고 ...


In [ ]:
test_df.head()

,sentence_id,person,sentence,scene,context,target_speaker_ctx,other_speaker_ctx,scene_sents,preprocessed
0,10,,"제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다. 음악소리. 경찰의날, 행...",S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",,,,제가 집안일땜에 잠깐 나와 있거든요 시간안에 가겠습니다 음악소리 경찰의날 행사 단상...
1,11,,"계약할 서류는 태연희실장 통해서 보내 드렸는데 확인해 보셨죠? 예, 예. 음악소리....",S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.,,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.,계약할 서류는 태연희실장 통해서 보내 드렸는데 확인해 보셨죠 예 예 음악소리 경찰의...
2,12,,"김순경 경사님 사모님 아니십니까? 음악소리. 경찰의날, 행사 단상차려진. 정복 예복...",S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다. 계약할 서류는 태연희실...,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.계약할 서류는 태연희실장...,김순경 경사님 사모님 아니십니까 음악소리 경찰의날 행사 단상차려진 정복 예복차려입은...
3,13,,"아 예. 음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. ...",S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다. 계약할 서류는 태연희실...,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.계약할 서류는 태연희실장...,아 예 음악소리 경찰의날 행사 단상차려진 정복 예복차려입은 경찰들 앉았고 일반 사람...
4,14,,"저 기억하시죠? 00지구대에서 김경사님 모시고 근무했었습니다. 음악소리. 경찰의날,...",S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",김순경 경사님 사모님 아니십니까?,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다. 계약할 서류는 태연희실...,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.계약할 서류는 태연희실장...,저 기억하시죠 지구대에서 김경사님 모시고 근무했었습니다 음악소리 경찰의날 행사 단상...


In [ ]:
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

# X, y = make_classification(n_samples=10000, n_features=5, weights=[0.99], flip_y=0)
# print(Counter(y))
# print(X)
# print(y)


sentence = train_df["preprocessed"].values.tolist()
#sentence = train_df["sentence"].values.tolist()
label = train_df["label"].values.tolist()
print(label)
sentence_arr = []
for sent in sentence:
  sentence_arr.append([sent])

# X_under, y_under = RandomUnderSampler(random_state=0).fit_resample(sentence_arr, label)
# print(Counter(y_under))

X_under, y_under = RandomOverSampler(random_state=0).fit_resample(sentence_arr, label)
print(Counter(y_under))

X_under_arr = []
for X in X_under:
  X_under_arr.extend(X)

#print(X_under_arr)
train_df_under = pd.DataFrame()
train_df_under["sentence"] = X_under_arr
train_df_under["label"] = y_under


['dysphoria', 'dysphoria', 'neutral', 'euphoria', 'dysphoria', 'neutral', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'neutral', 'dysphoria', 'neutral', 'dysphoria', 'euphoria', 'euphoria', 'euphoria', 'dysphoria', 'neutral', 'dysphoria', 'euphoria', 'dysphoria', 'dysphoria', 'euphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'neutral', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'neutral', 'dysphoria', 'dysphoria', 'euphoria', 'euphoria', 'euphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'euphoria', 'euphoria', 'neutral', 'neutral', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'euphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'dysphoria', 'neutral', 'euphoria', 'dysphoria', 'dysp

In [ ]:
train_df_under.head(40)

,sentence,label
0,"엄마 왜 종남이만 많이 줘? 내거야. 상태보면 종남접시에 좀더 많이 담긴 자,...",dysphoria
1,또 그놈의 잔소리. 마지못해 반찬꺼내 상차리는 과자. 들오는 순경. 뒤따라 현찰 ...,dysphoria
2,너한테 무슨말 안하디? 저녁먹던중. 들어와 앉는 범인부영 언니 먹을것좀 해줘야겠다...,neutral
3,"아, 오빠만 보면 삘이팍! 온다구요. 할수없이 청난옆에 앉는 건강. 좋아서 얼른 팔...",euphoria
4,태워주긴 뭘 태워줘? 걸어서 가. 부영이 앞으로 언니 신경 안쓰이게 니할일 딱딱 하...,dysphoria
5,대개 애인은 1번으로 해놓거든요. 찾아봐라. 몇번인지.,neutral
6,뭣이여? 속은 내가 터지는디 물은 왜 니가 쳐마셔? 나도 놀랬다. 이싸가지가 느...,dysphoria
7,"너 뭐야? 분노로 악 지르며 마구 부수는데, 건물 앞에서 뛰쳐 나오는 박사장과 어깨...",dysphoria
8,한번도 경찰이 된걸 후회한적 없지만 지금 이순간만큼은 제가 경찰이라는게 한이 됩니다...,dysphoria
9,"도련님 걱정마세요. 제가 삘이 팍! 가는 여자로 중매설게요. 어머, 그게 정말이...",dysphoria


In [ ]:
def get_pipe(model, model_name: str) -> Pipeline:
    "TfidfVectorizer와 모델을 연결한 파이프라인을 반환하는 함수"
    tfidf = TfidfVectorizer(analyzer="char", ngram_range=(1, 2)) # 1,3 -> 1,2 최고 성능 갱신
    pipe = Pipeline([
        ("tfidf", tfidf),
        (model_name, model)
    ])
    return pipe

In [ ]:
from sklearn.metrics import f1_score
def return_kfold_accuarcy(model, k: int = 5) -> float:
    "모델을 입력받아 KFold 예측 후 accuracy score를 반환하는 함수"
    kfold = StratifiedKFold(k, shuffle=True, random_state=42)
    result = []
    n_iter = 0
    for train_idx, test_idx in kfold.split(train_df["sentence"], train_df["label"]):
        n_iter = n_iter+1
        label_train = train_df['label'].iloc[train_idx]
        label_test = train_df['label'].iloc[test_idx]

        train, val = train_df.iloc[train_idx], train_df.iloc[test_idx]
        model.fit(train["sentence"], train["label"])
        pred = model.predict(val["sentence"])
        acc = accuracy_score(val["label"], pred)
        acc2 = f1_score(val["label"], pred, average='macro')
        print('################# ################# : {}'.format(n_iter))
        print('교차 검증 정확도 : {}'.format(acc))
        print('교차 검증 f1 정확도 : {}'.format(acc2))
        print(' 학습 레이블 분포(label_train) : \n', label_train.value_counts())
        print(' 테스트 레이블 분포(label_test) : \n', label_test.value_counts())
        print('\n')

        result.append(acc2)

    return np.mean(result)

In [ ]:
def return_kfold_accuarcy_under(model, k: int = 5) -> float:
    "모델을 입력받아 KFold 예측 후 accuracy score를 반환하는 함수"
    kfold = StratifiedKFold(k, shuffle=True, random_state=42)
    result = []
    n_iter = 0
    for train_idx, test_idx in kfold.split(train_df_under["sentence"], train_df_under["label"]):
        n_iter = n_iter+1
        label_train = train_df_under['label'].iloc[train_idx]
        label_test = train_df_under['label'].iloc[test_idx]

        train, val = train_df_under.iloc[train_idx], train_df_under.iloc[test_idx]
        model.fit(train["sentence"], train["label"])
        pred = model.predict(val["sentence"])
        acc = accuracy_score(val["label"], pred)
        acc2 = f1_score(val["label"], pred, average='macro')
        print('################# ################# : {}'.format(n_iter))
        print('교차 검증 정확도 : {}'.format(acc))
        print('교차 검증 f1 정확도 : {}'.format(acc2))
        print(' 학습 레이블 분포(label_train) : \n', label_train.value_counts())
        print(' 테스트 레이블 분포(label_test) : \n', label_test.value_counts())
        print('\n')

        result.append(acc2)

    return np.mean(result)

In [ ]:
# 0.67에 대한 베스트 모델링
# model = naive_bayes, SGD, lgbm2 (n_estimators=3) // 주최측 기본 워드에 // oversampling 이었음
models = [
    ("naive_bayes", BernoulliNB()),
    ("SGD", SGDClassifier(loss='squared_loss',random_state=42, n_jobs=-1)),
    #("rfc", RandomForestClassifier(random_state=42, n_jobs=-1)),
    #("SVM", SVC(random_state=42)),
    #("ada", AdaBoostClassifier(random_state=42)),
    # ("lgbm", lgb.LGBMClassifier(random_state=42,boosting_type='gbdt', num_leaves=31, max_depth=- 1, learning_rate=0.1, n_estimators=300, device = "gpu")),
    ("lgbm2", LGBMClassifier(n_estimators=4, random_state=42)), # 3일때가 제일 좋음
    #("xgb", XGBClassifier(random_state=42)),
    # ("knc1", KNeighborsClassifier()),
    #("knc2", KNeighborsClassifier(n_neighbors=4))
]

model_pipes = [(name, get_pipe(model, name)) for name, model in models]

In [ ]:
table = Table(title="Model Comparison Table")
table.add_column("Model Name", justify="left", style="green")
table.add_column("Accuracy", justify="right")

for model_name, model in tqdm(model_pipes, leave=False):
    #acc = return_kfold_accuarcy(model)
    acc = return_kfold_accuarcy_under(model)
    table.add_row(model_name, f"{acc:0.3f}")

rich.print(table)

  0%|          | 0/3 [00:00<?, ?it/s]

################# ################# : 1
교차 검증 정확도 : 0.6574585635359116
교차 검증 f1 정확도 : 0.6583992865608784
 학습 레이블 분포(label_train) : 
 dysphoria    2172
neutral      2172
euphoria     2172
Name: label, dtype: int64
 테스트 레이블 분포(label_test) : 
 dysphoria    543
euphoria     543
neutral      543
Name: label, dtype: int64


################# ################# : 2
교차 검증 정확도 : 0.6715776550030693
교차 검증 f1 정확도 : 0.6710558720657201
 학습 레이블 분포(label_train) : 
 dysphoria    2172
neutral      2172
euphoria     2172
Name: label, dtype: int64
 테스트 레이블 분포(label_test) : 
 dysphoria    543
neutral      543
euphoria     543
Name: label, dtype: int64


################# ################# : 3
교차 검증 정확도 : 0.6838551258440762
교차 검증 f1 정확도 : 0.6864898178148441
 학습 레이블 분포(label_train) : 
 dysphoria    2172
neutral      2172
euphoria     2172
Name: label, dtype: int64
 테스트 레이블 분포(label_test) : 
 dysphoria    543
neutral      543
euphoria     543
Name: label, dtype: int64


################# ################# : 4


  Model Comparison Table  
┏━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Model Name  ┃ Accuracy ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ naive_bayes │    0.667 │
│ SGD         │    0.898 │
│ lgbm2       │    0.750 │
└─────────────┴──────────┘

In [ ]:
from sklearn.ensemble import StackingClassifier

stack_models = [(name, get_pipe(model, name)) for name, model in models]

stacking = StackingClassifier(stack_models)
#acc = return_kfold_accuarcy(stacking)
#acc = return_kfold_accuarcy_under(stacking)
#rich.print(acc)

In [ ]:
#stacking.fit(train_df['sentence'], train_df['label'])
stacking.fit(train_df_under['sentence'], train_df_under['label'])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:169: FutureWarning: The loss 'squared_loss' was deprecated in v1.0 and will be removed in version 1.2. Use `loss='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:700: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:169: FutureWarning: The loss 'squared_loss' was deprecated in v1.0 and will be removed in version 1.2. Use `loss='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:700: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,
/usr/local/lib/python3.7

StackingClassifier(estimators=[('naive_bayes',
                                Pipeline(steps=[('tfidf',
                                                 TfidfVectorizer(analyzer='char',
                                                                 ngram_range=(1,
                                                                              2))),
                                                ('naive_bayes',
                                                 BernoulliNB())])),
                               ('SGD',
                                Pipeline(steps=[('tfidf',
                                                 TfidfVectorizer(analyzer='char',
                                                                 ngram_range=(1,
                                                                              2))),
                                                ('SGD',
                                                 SGDClassifier(loss='squared_loss',
                                

In [ ]:
#submission_pred = stacking.predict(test_df['sentence'])
submission_pred = stacking.predict(test_df['preprocessed'])

In [ ]:
submission_df = pd.DataFrame()
submission_df["Id"] = test_df['sentence_id']
submission_df["Predicted"] = submission_pred
submission_df.to_csv(f'/content/gdrive/MyDrive/kerc/logs/submission_over_stacking_3model_validation_sgb_stopword_nothing_tf_idf2_only_public.csv', index=False)
print(f"Saved 'logs/submission_over_stacking_3model_validation_sgb_stopword_nothing_tf_idf2_only_public.csv")

Saved 'logs/submission_over_stacking_3model_validation_sgb_stopword_nothing_tf_idf2_only_public.csv


In [ ]:
print(submission_df["Predicted"].value_counts())

dysphoria    1786
neutral       391
euphoria      389
Name: Predicted, dtype: int64


In [ ]:
validation_df.head()

,sentence_id,person,sentence,scene,context,target_speaker_ctx,other_speaker_ctx,scene_sents,label,preprocessed
0,4686,,같이 가. 현찰 이미 좀 취한 그래. 그래서 니 별명이 딱지대장이었잖아. 어떻게 ...,S1504,현찰 이미 좀 취한,그래. 그래서 니 별명이 딱지대장이었잖아. 어떻게 그렇게 딱지를 잘치는지. 맨날 ...,"내가? 맞어, 그랬어. 생각난다. 친구들하고 돈으로 바꿨지. 저축. 너두 같이 줏...",내가?그래. 그래서 니 별명이 딱지대장이었잖아. 어떻게 그렇게 딱지를 잘치는지.맞...,euphoria,같이 가 현찰 이미 좀 취한 그래 그래서 니 별명이 딱지대장이었잖아 어떻게 그렇게...
1,3158,,"이런식으로 가르치니 발전이 없죠. 격투기라는게 말 그대로 싸우는 기술인데, 실전에 ...",S1060,"관장안고 누운채 땀뻘뻘 흘리며 애쓰고 있는 재수. 서로 재는데, 이상한테 맞던 장면...",지금 하는 중이거든요.,"버티지만 말고 공격 들어오세요. 이런식으로 하니까 발전이 없는거,","버티지만 말고 공격 들어오세요.지금 하는 중이거든요.이런식으로 하니까 발전이 없는거,",dysphoria,이런식으로 가르치니 발전이 없죠 격투기라는게 말 그대로 싸우는 기술인데 실전에 써먹...
2,3969,,너 괜찮니? 신문보는 범인. 찻잔놓는 솔이. 이층에서 어영의 노랫소리. 슥보는 범인...,S1326,신문보는 범인. 찻잔놓는 솔이. 이층에서 어영의 노랫소리. 슥보는 범인,벌써 나가게?,먼일인지 아주 신바람이 났시유. 예 아부지. 일찍 일어나서 운동도 하고 책상 정리...,먼일인지 아주 신바람이 났시유.벌써 나가게?예 아부지. 일찍 일어나서 운동도 하고...,euphoria,너 괜찮니 신문보는 범인 찻잔놓는 솔이 이층에서 어영의 노랫소리 슥보는 범인 벌써...
3,340,,딸이 둘인가 있었는데...찾을려고 맘먹으면 경찰이 왜 못찾겠냐만은... 남들은 ...,S0202,,"남들은 고시패스하면 편한길부터 갈려고 하는데, 나중에라도 후회하지 않겠냐? 니엄마...","제가 사시 본건 경찰일을 더 잘하기 위해서지, 다른뜻 없습니다. 솔직히 합격하고 ...","남들은 고시패스하면 편한길부터 갈려고 하는데, 나중에라도 후회하지 않겠냐?제가 사...",dysphoria,딸이 둘인가 있었는데찾을려고 맘먹으면 경찰이 왜 못찾겠냐만은 남들은 고시패스하면...
4,8711,,사채 썼다며. 아주 떡대같은 놈들이 찾아와서 협박하고 가드라. 모르는 소리야? ...,S2602,,나 지금 최대한 인내하고 있다. 지금부터 내가 묻는말에 거짓말 한개없이 솔직하게 ...,"오빠, 무슨 일있어? 아까 전화 목소리도 안좋던데. 무슨 빚? 나 빚같은거 없어....","오빠, 무슨 일있어? 아까 전화 목소리도 안좋던데.나 지금 최대한 인내하고 있다....",dysphoria,사채 썼다며 아주 떡대같은 놈들이 찾아와서 협박하고 가드라 모르는 소리야 나 지...


In [ ]:
from sklearn.metrics import f1_score
validation_pred = stacking.predict(validation_df['preprocessed'])
#validation_pred = stacking.predict(validation_df['sentence'])
print(validation_pred)
acc2 = f1_score(validation_df["label"], validation_pred, average='macro')
print(acc2)

['dysphoria' 'dysphoria' 'euphoria' ... 'neutral' 'dysphoria' 'dysphoria']
0.7102969162705145


0.7367796661075262 // 4일때 best

0.7325731985111089 // 5일때 떨어짐

0.7312302746212364 // 2일때

0.7335518645891691 // lgb :4 고정, sgb : 로스 부분 수정중

In [ ]:
from collections import Counter
print(Counter(validation_pred))

Counter({'dysphoria': 1699, 'euphoria': 814, 'neutral': 423})


In [ ]:
submission_df2 = pd.DataFrame()
submission_df2["Id"] = validation_df["sentence_id"]
submission_df2["Sentence"] = validation_df["sentence"]
submission_df2["preprocessed"] = validation_df['preprocessed']
submission_df2["Predicted"] = validation_pred
submission_df2["Y_Hat"] = validation_df["label"]
submission_df2.to_csv(f'/content/gdrive/MyDrive/kerc/logs/validation_over_stacking_3model_pre_validation_tf_idf.csv', index=False, encoding='utf-8-sig')
print(f"Saved 'logs/validation_over_stacking_3model_pre_validation_tf_idf.csv")

Saved 'logs/validation_over_stacking_3model_pre_validation_tf_idf.csv


In [ ]:
submission_df2.head(60000)

,Id,Sentence,preprocessed,Predicted,Y_Hat
0,4686,같이 가. 현찰 이미 좀 취한 그래. 그래서 니 별명이 딱지대장이었잖아. 어떻게 ...,같이 가 현찰 이미 좀 취한 그래 그래서 니 별명이 딱지대장이었잖아 어떻게 그렇게...,euphoria,euphoria
1,3158,"이런식으로 가르치니 발전이 없죠. 격투기라는게 말 그대로 싸우는 기술인데, 실전에 ...",이런식으로 가르치니 발전이 없죠 격투기라는게 말 그대로 싸우는 기술인데 실전에 써먹...,dysphoria,dysphoria
2,3969,너 괜찮니? 신문보는 범인. 찻잔놓는 솔이. 이층에서 어영의 노랫소리. 슥보는 범인...,너 괜찮니 신문보는 범인 찻잔놓는 솔이 이층에서 어영의 노랫소리 슥보는 범인 벌써...,euphoria,euphoria
3,340,딸이 둘인가 있었는데...찾을려고 맘먹으면 경찰이 왜 못찾겠냐만은... 남들은 ...,딸이 둘인가 있었는데찾을려고 맘먹으면 경찰이 왜 못찾겠냐만은 남들은 고시패스하면...,dysphoria,dysphoria
4,8711,사채 썼다며. 아주 떡대같은 놈들이 찾아와서 협박하고 가드라. 모르는 소리야? ...,사채 썼다며 아주 떡대같은 놈들이 찾아와서 협박하고 가드라 모르는 소리야 나 지...,dysphoria,dysphoria
...,...,...,...,...,...
2931,10489,알았어. 뭐하러 쓸데없는 짓은 해? 만나서 얘기해. 내가 그쪽으로 갈게. 쓸데...,알았어 뭐하러 쓸데없는 짓은 해 만나서 얘기해 내가 그쪽으로 갈게 쓸데없는 짓...,dysphoria,dysphoria
2932,3228,그래? 그럼 내가 올라가서 음식이라도 좀 해줄까? 대체 무슨 일인가? 무슨 일인...,그래 그럼 내가 올라가서 음식이라도 좀 해줄까 대체 무슨 일인가 무슨 일인데 자...,dysphoria,dysphoria
2933,3124,무슨 일로 집합시켰어? 기다리고선 건강. 급히 오는 현찰 이상 군기들이 왕창 ...,무슨 일로 집합시켰어 기다리고선 건강 급히 오는 현찰 이상 군기들이 왕창 빠졌...,neutral,neutral
2934,10822,"부킹해서 첨만난게 아냐? 어영. 친구2 아줌마1,2 둘러싼 왜 아직 식을 안하...",부킹해서 첨만난게 아냐 어영 친구 아줌마 둘러싼 왜 아직 식을 안하지 야야 신...,dysphoria,dysphoria


dysphoria    2257
euphoria      231
neutral        78 // preprocessed (navie + sg)

dysphoria    2291
euphoria      196
neutral        79 // sentence (navie + sg)

dysphoria    2238
euphoria      235
neutral        93 // sentence (navie + sg + lgb)

dysphoria    2202
euphoria      263
neutral       101 // preprocessed (navie + sg + lgb)

dysphoria    2254
euphoria      210
neutral       102 // preprocessed (navie + sg + lgb + knc2)

dysphoria    2196
euphoria      272
neutral        98 // preprocessed (navie + svm + lgb) -> 30분 걸림

dysphoria    2253
euphoria      231
neutral        82 // sentence (navie + svm + lgb)

------
sgwannabe dataset 스코어
Counter({'dysphoria': 1757, 'euphoria': 793, 'neutral': 386}) // 0.7257815832919629 -> 실제 : dysphoria    1929
euphoria      568
neutral        69 --> 0.68667
